# In this code, we will build a Linear Regression model to estimate income, and will build a marketing strategty based on the model's output (i.e. estimated income).

### **Problem Statement:** A producer of luxury products, need a model to target potential customers for marketing and advertising purposes. Based on the previous marketing campaigns, we have the following information:
1.   It costs company 10 Dollars to advertise to each customer.
2.   Company makes 500 Dollars profit on each product sold.
3.   If company advertises to a customer, they will buy the product if and only if they make more than 170,000 Dollars per year.
4.   No transaction happens without advertisement.  


### **ML Model:** Company has data on income of some customers. Modeling team will use this data to build an Income Estimation Model.

### **Strategy:** Output of the above model will be used to define an advertisement strategy / threshold. Company will advertise to customers with Estimated Income (model's output) higher than the threshold.

In [1]:
# import required libraries

import pandas as pd
import numpy as np

In [2]:
# this step is to read data on Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Data shared on eLearning
income_data=pd.read_csv("drive/My Drive/Income_Data.csv")

# First: Data Exploration - Getting to Know our Data

In an actual project, this step would be "Data Collection and Exploration". We assume data is already collected.

In [4]:
# Check number of rows (observations) and columns (features)
income_data.shape

(6702, 6)

In [5]:
# Look at the first few observations
income_data.head(5)

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28,Female,Master's,Data Analyst,3.0,65000.0
2,45,Male,PhD,Senior Manager,15.0,150000.0
3,36,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52,Male,Master's,Director,20.0,200000.0


In [6]:
# and last few observations, to check if something might have changed in between
income_data.tail(5)

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
6697,49,Female,PhD,Director of Marketing,20.0,200000.0
6698,32,Male,High School,Sales Associate,3.0,50000.0
6699,30,Female,Bachelor's Degree,Financial Manager,4.0,55000.0
6700,46,Male,Master's Degree,Marketing Manager,14.0,140000.0
6701,26,Female,High School,Sales Executive,1.0,35000.0


In [7]:
# take a look at the column names
income_data.columns

Index(['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience',
       'Salary'],
      dtype='object')

In [8]:
# check columns' data types
income_data.dtypes

Age                      int64
Gender                  object
Education Level         object
Job Title               object
Years of Experience    float64
Salary                 float64
dtype: object

In [ ]:
# check Summary Statistics for numerical fields
income_data.describe()

,Age,Years of Experience,Salary
count,6702.000000,6701.000000,6699.000000
mean,33.620859,8.095657,115326.964771
std,7.614633,6.057975,52786.183911
min,21.000000,0.000000,350.000000
25%,28.000000,3.000000,70000.000000
50%,32.000000,7.000000,115000.000000
75%,38.000000,12.000000,160000.000000
max,62.000000,34.000000,250000.000000


*Assignment 1: Generate other percentiles: P1, P5, P10, P90, P95, P99.*

# 2. Data Cleaning - remove unwanted features and observations

In [9]:
# Summary statistics shows there are 3 observations with missing income information. We often remove observations when Target variable is missing.

# double-check observations with missing target:
income_data[income_data.Salary.isna()].shape

# remove observations with missing Target.
income_data = income_data[income_data.Salary.notnull()]

# ALWAYS check the data after each step.
print (income_data.shape)
print(income_data[income_data.Salary.isna()].shape)

(6699, 6)
(0, 6)


In [10]:
# There are three categorical (non-numerical) features. To build a ML model, we need to convert all features to numerical.
# We will discuss this process in the next sessions. For now we just remove the categorical features.
income_data.drop(["Gender", "Education Level", "Job Title"], axis = 1, inplace = True)

# note inplace column. this is a parameter to set when you change something a DataFrame. If not, the changes will not be saved to the DF.

# check
income_data.columns

Index(['Age', 'Years of Experience', 'Salary'], dtype='object')

# 3. Data Processing

### Main data processing steps for linear models:
1. **One-Hot Ecoding**: i.e. converting categorical observations to numerical. For this model, we just removed categorical features, and will discuss this step in the next sessions.
2. **Outlier Treatment**: We can detect outliers by looking at summary statistics, and comparing P99 with Max and P1 with Min. A large difference indicates an outlier. Another way to detect outliers is by looking at the distribution (histogram) of the feature, and detect outliers optically. To treat outliers, one approach is to use P1 and P99 as floor and cap, respectively. i.e. replace any value less than P1 by P1 and replace any value higher than P99 by P99.

Here, I leave it on you to calculate P1 and P99 percentiles (in assignment 1), and in assignment 2 ask you to:

*Assignment 2: Cap (and floor) both features, to their 99th (and 1st) percentiles.*
3. **Feature Scaling**: Ideally we like all **Independent Variables** to have similar scales. If not, linear model will focus on features with larger scales, and will generate non-optimal results, because information in features with lower scale, will be lost.

Here, our (independent) features, "Age" and "Years of Experience" have close scales, so no feature scaling needed. We will discuss popular feature scaling techniques, in Neural Netwok session.
4. **Missing Value Imputation**: Missing values are missing information. Like about someone's Age. Machine expects **ONLY** numbers.
If missing happens in the target (dependent) variable, we often remove those observations. Unless limited data is available.

If missing value happens in independent variables, some popular approaches are as following:
*   Replace missing values of the feature, with average of non-missing observations
*   Replace missing values of the feature, with median of non-missing observations
*   With 0
*   With a very large or small number, like: 999,999 or -999,999. In order to distinguish these observations from non-missing values. These way, we can penalize lack of information. More on this, Neural Network session.

Summary statistics shows there is no missing in "Age", but there is one in "Years of Experience". It may have already be deleted (when we deleted missing Salary), but we will check. If not deleted, we will replace it with 0.

Some Machine Learning packages, handle missing values automatically, so you don't have to worry about how to impute missing values. More on this in Ensemble Models session.


We may also decide to delete all observations with any missing (in dependent, or independent variables), but often we would end up with too many exclusions, and we don't have unlimited data.


**Data Quantity** and **Computational Power** define limts in a Data Science project, *limits of our dreams regarding what machine can do*. Find a way to expand the **Optimization Boundary** in a field, beyond others can do, and you are on the path to have your own company, a data-driven start-up.

In this calss, we will discuss rules of boundary.

5. **No Multi-Collinearity**: Last step in data processing for a inear model is to check for Multi-Collinearity. Multi-Collinearity means some features are highly correlated. That makes the model non-robust; i.e. model would chang siginificantly sample from sample (different samples from the population).

In case of Multi-Collinearity, we keep only one feature among the ones with high correlation. What is high correlation? 70%, 80%, 90%, 99%??? It is a judgmental call, like many other steps in a ML project. Depends on data quantity, data structure and relationship between attributes, ...

Here, since we have only two independent variables and for simplicity, we don't check for multi-collinearity.

In [11]:
# missing value imputation
income_data[income_data["Years of Experience"].isna()].shape

# looks like the observation is gone.

(0, 3)

# 4. Model Training

We collected and explored the data, Cleaned the data, and performed data processing for a Linear model. Now we will train a Linear model.

We need a package that does that. Here we will use "statsmodels" package. It is a good package for Linear models.


In almost all ML packages that I know (like Linear, Decision Tree, Neural Network, XGBoost, Recommender Systems, K-Mean Clustering, ...), we do these three steps:

**1.   Define an Object from that package, and**

**2.   Set model's (object's) parameters, dependent variable, independent variables, ...**

**3.   Fit dependent variable (Y) on independent variables (Xs)**



In [13]:
import statsmodels.api as sm

In [14]:
# define the object, and set model's parameters
model = sm.OLS(income_data["Salary"], income_data[["Age", "Years of Experience"]])

# fit the model, and save the results in an object called "Income_Model"
Income_Model = model.fit()

In [ ]:
#  look at the model's results
Income_Model.summary()

# results show this model: Salary-Hat (Expected Salary Conditional on Age and Years of Experience) = 2284.47 * Age + 4612.94 * Years of Experience
# For example, Expected Salary for someone with 30 Years of Age and 2 Years of Experience = 2284.47 * 30 + 4612.94 * 2 = 77,759.98 Dollars annual income.

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.934
Model:                            OLS   Adj. R-squared (uncentered):              0.934
Method:                 Least Squares   F-statistic:                          4.749e+04
Date:                Sun, 27 Aug 2023   Prob (F-statistic):                        0.00
Time:                        22:01:07   Log-Likelihood:                         -79111.
No. Observations:                6699   AIC:                                  1.582e+05
Df Residuals:                    6697   BIC:                                  1.582e+05
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Age                  2284.4738     27.117     84.244      0.000    2231.315    2337.632
Years of Experience  4612.9411     92.449     49.897      0.000    4431.711    4794.171
==============================================================================
Omnibus:                      353.005   Durbin-Watson:                   0.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              412.581
Skew:                           0.577   Prob(JB):                     2.57e-90
Kurtosis:                       3.380   Cond. No.                         8.59
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# Note that the model had no intercept. In this package, to build a model with intercept, we need to add a column - all 1s - to the matrix
# of independent variables (Xs)

# One skill we need to excel in, is to find optimum, easy-friendly code to do different tasks. 50+ % of the code above, is by search.

# define the new column (of ones)
new_column = np.ones(income_data.shape[0])

# insert the new column at the beginning of Xs
Xs = income_data[["Age", "Years of Experience"]]
Xs.insert(loc=0, column='Intercept', value=new_column)

#check the results
Xs.head(2)

,Intercept,Age,Years of Experience
0,1.0,32,5.0
1,1.0,28,3.0


In [16]:
# Build the model with intercept
model = sm.OLS(income_data["Salary"], Xs)
Income_Model = model.fit()
Income_Model.summary()

# this time Expected Salary conditional on Age = 30 and Years of Experience = 2 = 100400 - 1748.39 * 30 + 9107.85 * 2 = 66.164 Dollars annual income.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     6556.
Date:                Sun, 27 Aug 2023   Prob (F-statistic):               0.00
Time:                        23:07:56   Log-Likelihood:                -78717.
No. Observations:                6699   AIC:                         1.574e+05
Df Residuals:                    6696   BIC:                         1.575e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.004e+05   3470.727     28.920      0.000    9.36e+04    1.07e+05
Age                 -1748.3891    141.772    -12.332      0.000   -2026.308   -1470.470
Years of Experience  9107.8462    178.201     51.110      0.000    8758.515    9457.177
==============================================================================
Omnibus:                      121.253   Durbin-Watson:                   0.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.117
Skew:                           0.255   Prob(JB):                     1.53e-33
Kurtosis:                       3.530   Cond. No.                         331.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 5. Define Strategy (threshold)

*Assignment 3. Try to understand the dollowing code, that finds the optimum threshold. Review Profit and Loss assumptions at the beginning of the code.*

In [20]:
# add predictions
income_data["Estimated Salary"] = Income_Model.predict(Xs)

# check
income_data.tail(2)

,Age,Years of Experience,Salary,Estimated Salary
6700,46,14.0,140000.0,147458.327881
6701,26,1.0,35000.0,64024.109134


In [21]:
# check distribution of model's output (estimated income)
income_data[["Salary", "Estimated Salary"]].describe()

# Notice the mean of both columns is exactly the same. Try to prove, mathematically, that this result is expected.

,Salary,Estimated Salary
count,6699.000000,6699.000000
mean,115326.964771,115326.964771
std,52786.183911,42946.767289
min,350.000000,51785.385480
25%,70000.000000,80491.412443
50%,115000.000000,106798.363421
75%,160000.000000,142863.847492
max,250000.000000,305137.804598


In [60]:
# define a function that calculates profit for any threshold.
# Note, company would advertise to those with estimated income higher than threshold.

def threshold_profit (data, Actual_Salary_Column, Estimated_Salary_Column, threshold):
  data_threshold_profit = data.copy()
  data_threshold_profit["advertising cost"] = 0
  data_threshold_profit["profit"] = 0
  data_threshold_profit['advertising cost'] = np.where(data_threshold_profit['Estimated Salary'] > threshold, 10, 0)
  data_threshold_profit['profit'] = np.where(data_threshold_profit['Salary'] > 170000,
                                             np.where(data_threshold_profit['Estimated Salary'] > threshold, 500, 0), 0)
  return (data_threshold_profit.profit.sum() - data_threshold_profit["advertising cost"].sum())



print (threshold_profit (income_data.head(5), "Salary", "Estimated Salary", 100000))

470


In [46]:
income_data.head(5)

,Age,Years of Experience,Salary,Estimated Salary
0,32,5.0,90000.0,89965.159378
1,28,3.0,65000.0,78743.023350
2,45,15.0,150000.0,158314.563176
3,36,7.0,60000.0,101187.295407
4,52,20.0,200000.0,191615.070528


In [86]:
# calcuate profit for a few thresholds and find the best
profit_results = pd.DataFrame(columns = ["Threshold", "Profit"])

row_number = 0
for threshold in range (10000, 300000, 10000):
  profit_results.loc[row_number, "Threshold"] = threshold
  profit_results.loc[row_number, "Profit"] = threshold_profit (income_data, "Salary", "Estimated Salary", threshold)
  row_number = row_number + 1

best_threshold = profit_results[profit_results.Profit == profit_results.Profit.max()]["Threshold"].to_string(index = False)
print ("The optimum Threshold is ", best_threshold)

The optimum Threshold is  100000
